# 建立一个工作流

## 导入一些代码

In [32]:
from llama_index.llms.glm import GLM
llm = GLM(temperature=0.7)

# from llama_index.llms.deepseekai import DeepSeek

In [33]:
from llama_index.core.tools import FunctionTool
from Embedding import BGEEbedding
from llama_index.core.tools import ToolMetadata,RetrieverTool
import logging
from load_from_chroma import get_index_from_chroma
from llama_index.core.agent import AgentRunner

logging.getLogger("agent_demo").setLevel(logging.INFO)
embd_model = BGEEbedding()


In [34]:
chroma_index = get_index_from_chroma(path=".\\LawDb",collection="laws",embedding_model=embd_model)

In [4]:
# a.retrieve("""杀人""")

In [44]:
import datetime


retriver_tool = RetrieverTool(
    retriever=chroma_index.as_retriever(),
    metadata=ToolMetadata(
        name="law_tools",
        description="""
             用户询问关于法律的任何信息时，调用此工具。
            """,
    ),
)
def get_now():
    """获取当前的年份，月份，日期和时间"""
    return datetime.datetime.now()


def multiply(a: int, b: int) -> int:
    """计算a和b的乘积并返回结果"""
    return a * b


def add(a: int, b: int) -> int:
    """计算a和b的和并返回结果"""
    return a + b
multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
get_local_time = FunctionTool.from_defaults(fn=get_now)

retriever_tool = RetrieverTool(
    retriever=chroma_index.as_retriever(),
    metadata=ToolMetadata(
        name="law_tools",
        description="""
             用户询问关于法律的任何信息时，调用此工具。
            """,
    ),
)

In [45]:
agent = AgentRunner.from_llm(
    [multiply_tool, add_tool, retriever_tool, get_local_time], llm=llm, verbose=True
)

In [46]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

In [47]:
class Flow(Workflow):
    @step
    async def step(self,ev:StartEvent) -> StopEvent:
        question = ev.get("input")
        responce = agent.chat(question)
        return StopEvent(result=str(responce))


In [48]:
w = Flow(timeout=10, verbose=False)
result = await w.run(input = """杀人犯什么罪？""")
print(result)

> Running step f598ce2c-3d66-4f70-bc2b-dcb7690f169f. Step input: 杀人犯什么罪？
Thought: The user is asking a legal question, so I should use the law_tools tool to get the answer.
Action: law_tools
Action Input: {'input': '杀人犯什么罪？'}
Observation: file_path = d:\code\law_llama_system\Law-Book\7-刑法\刑法.md
第四章 侵犯公民人身权利、民主权利罪

第二百三十二条 故意杀人的，处死刑、无期徒刑或者十年以上有期徒刑；情节较轻的，处三年以上十年以下有期徒刑。

第二百三十三条 过失致人死亡的，处三年以上七年以下有期徒刑；情节较轻的，处三年以下有期徒刑。本法另有规定的，依照规定。

第二百三十四条 故意伤害他人身体的，处三年以下有期徒刑、拘役或者管制。

犯前款罪，致人重伤的，处三年以上十年以下有期徒刑；致人死亡或者以特别残忍手段致人重伤造成严重残疾的，处十年以上有期徒刑、无期徒刑或者死刑。本法另有规定的，依照规定。

第二百三十四条之一 组织他人出卖人体器官的，处五年以下有期徒刑，并处罚金；情节严重的，处五年以上有期徒刑，并处罚金或者没收财产。

未经本人同意摘取其器官，或者摘取不满十八周岁的人的器官，或者强迫、欺骗他人捐献器官的，依照本法第二百三十四条、第二百三十二条的规定定罪处罚。

file_path = d:\code\law_llama_system\Law-Book\7-刑法\刑法.md
第三百零一条 聚众进行淫乱活动的，对首要分子或者多次参加的，处五年以下有期徒刑、拘役或者管制。

引诱未成年人参加聚众淫乱活动的，依照前款的规定从重处罚。

第三百零二条 盗窃、侮辱、故意毁坏尸体、尸骨、骨灰的，处三年以下有期徒刑、拘役或者管制。

第三百零三条 以营利为目的，聚众赌博或者以赌博为业的，处三年以下有期徒刑、拘役或者管制，并处罚金。

开设赌场的，处五年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处五年以上十年以下有期徒刑，并处罚金。

组织中华人民共和国公民参与国（境）外赌博，

In [40]:
print(result)

杀人犯的罪名是故意杀人罪，根据我国刑法的规定，故意杀人的，处死刑、无期徒刑或者十年以上有期徒刑；情节较轻的，处三年以上十年以下有期徒刑。


In [49]:
agent.chat("""现在是什么时候，距离10月1日还有几天？""")

> Running step 1787590b-4516-4afd-b617-886dc1a36fde. Step input: 现在是什么时候，距离10月1日还有几天？
Thought: (Implicit) I can answer without any more tools!
Answer: get_now
{}


AgentChatResponse(response='get_now\n{}', sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

In [50]:
agent.chat("""计算10×6,同时1+281313的值是多少？""")

> Running step 470c2a24-597f-4da5-82d1-49686f3de443. Step input: 计算10×6,同时1+281313的值是多少？
Thought: (Implicit) I can answer without any more tools!
Answer: multiply
{"a": 10, "b": 6}


AgentChatResponse(response='multiply\n{"a": 10, "b": 6}', sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)